<a href="https://colab.research.google.com/github/shivangsingh26/FL-BC-BTP/blob/master/FedAvg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow federated
!pip install gdown

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import zipfile

# Unzip the datasets
zip_path = '/content/drive/My Drive/BTP/non_iid.zip'
unzip_path = '/content/datasets'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

Mounted at /content/drive


In [ ]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_data(client_path):
    train_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        os.path.join(client_path, 'train'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    test_generator = test_datagen.flow_from_directory(
        os.path.join(client_path, 'test'),
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical'
    )

    return train_generator, test_generator

clients = ['/content/datasets/Non-iid datasets/non_iid_subset_1','/content/datasets/Non-iid datasets/non_iid_subset_2','/content/datasets/Non-iid datasets/non_iid_subset_3','/content/datasets/Non-iid datasets/non_iid_subset_4',]
data_paths = [os.path.join(unzip_path, client) for client in clients]

train_generators = []
test_generators = []
for path in data_paths:
    train_gen, test_gen = load_data(path)
    train_generators.append(train_gen)
    test_generators.append(test_gen)


Found 19012 images belonging to 19 classes.
Found 4748 images belonging to 19 classes.
Found 19185 images belonging to 19 classes.
Found 4791 images belonging to 19 classes.
Found 19703 images belonging to 19 classes.
Found 4921 images belonging to 19 classes.
Found 19530 images belonging to 19 classes.
Found 4878 images belonging to 19 classes.


In [ ]:
def create_model(num_classes=19):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
import numpy as np

# Define number of communication rounds
num_rounds = 5

# Initialize models for each client
client_models = [create_model(num_classes=19) for _ in range(4)]

# Function to aggregate model weights
def fed_avg(models):
    avg_model = create_model(num_classes=19)  # Use 19 classes for the aggregated model
    avg_weights = [np.zeros_like(weights) for weights in avg_model.get_weights()]

    # Sum the weights from all models
    for model in models:
        model_weights = model.get_weights()
        for i in range(len(avg_weights)):
            avg_weights[i] += model_weights[i]

    # Average the weights
    avg_weights = [weight / len(models) for weight in avg_weights]
    avg_model.set_weights(avg_weights)

    return avg_model

for round_num in range(num_rounds):
    print(f'Round {round_num+1}/{num_rounds}')

    # Train each client model on its local data
    for i in range(4):
        print(f'Training client {i+1}')
        client_models[i].fit(train_generators[i], epochs=1, validation_data=test_generators[i])

    # Aggregate the client models
    global_model = fed_avg(client_models)

    # Update client models with the aggregated global model weights
    for i in range(4):
        client_models[i].set_weights(global_model.get_weights())


9406464/9406464 [==============================] - 0s 0us/step
Round 1/5
Training client 1
595/595 [==============================] - 99s 155ms/step - loss: 0.7760 - accuracy: 0.7780 - val_loss: 0.2311 - val_accuracy: 0.9404
Training client 2
600/600 [==============================] - 97s 156ms/step - loss: 0.5699 - accuracy: 0.8396 - val_loss: 0.1738 - val_accuracy: 0.9497
Training client 3
616/616 [==============================] - 99s 155ms/step - loss: 0.7229 - accuracy: 0.7917 - val_loss: 0.2200 - val_accuracy: 0.9340
Training client 4
611/611 [==============================] - 95s 149ms/step - loss: 0.6625 - accuracy: 0.8072 - val_loss: 0.1853 - val_accuracy: 0.9506
Round 2/5
Training client 1
595/595 [==============================] - 88s 148ms/step - loss: 0.6971 - accuracy: 0.7981 - val_loss: 0.2742 - val_accuracy: 0.9221
Training client 2
600/600 [==============================] - 90s 149ms/step - loss: 0.5160 - accuracy: 0.8496 - val_loss: 0.1785 - val_accuracy: 0.9493
Train

In [ ]:
for i in range(4):
    loss, accuracy = global_model.evaluate(test_generators[i])
    print(f'Client {i+1} - Loss: {loss}, Accuracy: {accuracy}')

149/149 [==============================] - 19s 116ms/step - loss: 1.7646 - accuracy: 0.4183
Client 1 - Loss: 1.7645994424819946, Accuracy: 0.4182813763618469
150/150 [==============================] - 18s 123ms/step - loss: 1.8661 - accuracy: 0.4475
Client 2 - Loss: 1.866053819656372, Accuracy: 0.44750574231147766
154/154 [==============================] - 18s 118ms/step - loss: 1.9093 - accuracy: 0.4107
Client 3 - Loss: 1.9092901945114136, Accuracy: 0.4106888771057129
153/153 [==============================] - 11s 69ms/step - loss: 1.8952 - accuracy: 0.4428
Client 4 - Loss: 1.8951634168624878, Accuracy: 0.4428044259548187
